In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import xarray as xr
from src.era5_processing import era_read_file
from src.cams_processing import cams_read_file, cams_0p75_read_file
from src.odiac_processing import odiac_read_file
from src.modis_processing import modis_ndvi_read_file
from src.landscan_processing import landscan_read_file
from src.gfed_processing import gfed_read_file
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.models import load_model

In [ ]:
# Set the directories

cams_file = '/Volumes/My Passport/datasets/cams/xco2_0p75/cams_xco2_0p75.nc'
cams_dir = '/Volumes/My Passport/datasets/cams/latest'
era_dir = '/Volumes/My Passport/datasets/era_new/predicting_data'
odiac_dir = '/Volumes/My Passport/datasets/odiac/1km'
ndvi_dir = '/Volumes/My Passport/datasets/modis/MYD13C1v061'
landscan_dir = '/Volumes/My Passport/datasets/landscan'
gfed_dir = '/Volumes/My Passport/datasets/gfed/4.1'

output_dir = 'predictions'

In [ ]:
def doy(year):
    start_date = datetime.date(year, 1, 1)
    end_date = datetime.date(year, 12, 31)

    current_date = start_date
    date_list = []
    while current_date <= end_date:
        date_list.append(current_date.strftime("%Y-%m-%d"))
        current_date += datetime.timedelta(days=1)

    return date_list

In [ ]:
#Import model
model = load_model('xco2_dnn_model.h5')

new_lats = np.arange(-89.625, 90, 0.75)
new_lons = np.arange(-179.625, 180, 0.75)

days = doy(2006) # Set the year
for d in days:

    filename = f'predicted_xco2_{d}.nc'
    output_filename = os.path.join(output_dir, filename)

    # Check if the output file already exists
    if os.path.exists(output_filename):
        print(f"File already exists for {d}, skipping...")
        continue
        
    print('Predicting data for', d)
    odiac_ds = odiac_read_file(odiac_dir, d).rename(
        {'x': 'longitude', 'y': 'latitude'}).drop_vars(['band', 'spatial_ref'])
    odiac_interp = odiac_ds.interp(latitude=new_lats, longitude=new_lons,
                                   kwargs={"fill_value": "extrapolate"})
    ndvi_ds = modis_ndvi_read_file(ndvi_dir, d)
    ndvi_interp = ndvi_ds.interp(latitude=new_lats, longitude=new_lons,
                                 kwargs={"fill_value": "extrapolate"})
    ndvi_interp['ndvi'] = xr.where((ndvi_interp['ndvi'] < 0) | ndvi_interp['ndvi'].isnull(),
                                   -3000, ndvi_interp['ndvi'])
    landscan_ds = landscan_read_file(landscan_dir, d).rename(
        {'x': 'longitude', 'y': 'latitude'}).drop_vars(['band', 'spatial_ref'])
    landscan_interp = landscan_ds.interp(latitude=new_lats, longitude=new_lons,
                                         kwargs={"fill_value": "extrapolate"})
    landscan_interp['landscan'] = xr.where((landscan_interp['landscan'] < 0) | landscan_interp['landscan'].isnull(),
                                           0, landscan_interp['landscan'])
    gfed_ds = gfed_read_file(gfed_dir, d)
    gfed_interp = gfed_ds.interp(latitude=new_lats, longitude=new_lons,
                                 kwargs={"fill_value": "extrapolate"})
    
    predicted_df = pd.DataFrame()
    cams_tmp = cams_0p75_read_file(cams_file, d)
    for cams_time in cams_tmp.time.values:
        era_ds = era_read_file(era_dir, d).sel(time=cams_time).drop_vars('time')
        era_interp = era_ds.interp(latitude=new_lats, longitude=new_lons,
                                   kwargs={"fill_value": "extrapolate"})
        cams_ds = (cams_tmp).sel(time=cams_time).drop_vars('time')
        cams_interp = cams_ds.interp(latitude=new_lats, longitude=new_lons,
                                     kwargs={"fill_value": "extrapolate"})
        cams2_ds = cams_read_file(cams_dir, d).sel(time=cams_time).drop_vars('time')
        cams2_ds_interp = cams2_ds.interp(latitude=new_lats, longitude=new_lons,
                                          kwargs={"fill_value": "extrapolate"})
        #cams2_ds_interp.drop_duplicates(inplace=True)
        cams2_ds_interp['cams2'] = cams2_ds_interp['cams2'] * 1000000

        predicting_ds = xr.merge([era_interp, cams_interp, cams2_ds_interp, odiac_interp, ndvi_interp, landscan_interp, gfed_interp])
        predicting_df = predicting_ds.to_dataframe().reset_index()
        input_df_tmp = predicting_df[['u10', 'v10', 'cams', 'cams2', 'odiac', 'ndvi', 'landscan', 'gfed']]

        scaler = MinMaxScaler()
        input_df = scaler.fit_transform(input_df_tmp)
        
        print('Predicting', cams_time)
        prediction = model.predict(input_df).flatten()
        predicted_df_tmp = pd.DataFrame({
            'longitude': predicting_df['longitude'],
            'latitude': predicting_df['latitude'],
            'time': cams_time,
            'XCO2': prediction})
        
        predicted_df = pd.concat([predicted_df, predicted_df_tmp], ignore_index=True)
    
    predicted_ds = xr.Dataset.from_dataframe(predicted_df.set_index(['time', 'latitude', 'longitude']))
    predicted_ds['XCO2'].attrs['units'] = 'ppm'
    predicted_ds['XCO2'].attrs['long_name'] = 'Column-averaged dry-air mole fraction of CO2'
    predicted_ds['time'].attrs['long_name'] = 'Time'
    predicted_ds['longitude'].attrs['units'] = 'degrees_east'
    predicted_ds['longitude'].attrs['long_name'] = 'Longitude'
    predicted_ds['latitude'].attrs['units'] = 'degrees_north'
    predicted_ds['latitude'].attrs['long_name'] = 'Latitude'

    output_filename = os.path.join(output_dir, filename)
    predicted_ds.to_netcdf(output_filename)